In [ ]:
import numpy as num
import pandas as pd 
import nltk
import inflect
import re,unicodedata
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
#nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
#nltk.download('averaged_perceptron_tagger')
from sklearn.ensemble import RandomForestClassifier
from sklearn import naive_bayes
import time

start_time = time.time()

In [ ]:
train = pd.read_csv("kdnuggetstrain.tsv",na_filter=False,skip_blank_lines=True,header=0, \
                    delimiter="\t", quoting=3)        
train.columns.values
x=num.array([id,"sentiment","review"], 
           dtype=object) 

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "can not","can't've": "can not have","'cause": "because",
  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not",
  "doesn't": "does not","don't": "do not","hadn't": "had not","he'll": "he will",  "he's": "he is",
  "hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would",
  "he'd've": "he would have","he'll've": "he will have","how'd": "how did","how'd'y": "how do you",
  "how'll": "how will",  "how's": "how is",  "I'd": "I would",  "I'd've": "I would have",
  "I'll": "I will",  "I'll've": "I will have",  "I'm": "I am",  "I've": "I have","i've": "i have",
  "isn't": "is not",  "it'd": "it had",  "it'd've": "it would have",  "it'll": "it will",
  "it'll've": "it will have",  "it's": "it is",  "let's": "let us",  "ma'am": "madam",
  "mayn't": "may not",  "might've": "might have",  "mightn't": "might not",  "mightn't've": "might not have",
  "must've": "must have",  "mustn't": "must not",  "mustn't've": "must not have",  "needn't": "need not",
  "needn't've": "need not have",  "o'clock": "of the clock",  "oughtn't": "ought not",  "oughtn't've": "ought not have",
  "shan't": "shall not",  "sha'n't": "shall not",  "shan't've": "shall not have",  "she'd": "she would",
  "she'd've": "she would have",  "she'll": "she will",  "she'll've": "she will have",
  "she's": "she is",  "should've": "should have", "what'll": "what will",  "what'll've": "what will have",
  "shouldn't": "should not",  "shouldn't've": "should not have",  "so've": "so have",  "so's": "so is",
  "that'd": "that would",  "that'd've": "that would have",  "that's": "that is",  "there'd": "there had",
  "there'd've": "there would have",  "there's": "there is",  "they'd": "they would",  "they'd've": "they would have",
  "they'll": "they will",  "they'll've": "they will have",  "they're": "they are",  "they've": "they have",
  "to've": "to have",  "wasn't": "was not",  "we'd": "we had",  "we'd've": "we would have",
  "we'll": "we will",  "we'll've": "we will have",  "we're": "we are",  "we've": "we have",  "weren't": "were not",
   "what're": "what are",  "what's": "what is",  "what've": "what have",  "when's": "when is",
  "when've": "when have",  "where'd": "where did",  "where's": "where is",
  "where've": "where have",  "who'll": "who will",  "who'll've": "who will have",
  "who's": "who is",  "who've": "who have",  "why's": "why is",
  "why've": "why have",  "will've": "will have",  "won't": "will not",  "won't've": "will not have",  "would've": "would have",
  "wouldn't": "would not",  "wouldn't've": "would not have",  "y'all": "you all",  "y'alls": "you alls",
  "y'all'd": "you all would",  "y'all'd've": "you all would have",  "y'all're": "you all are",
  "y'all've": "you all have",  "you'd": "you had",  "you'd've": "you would have",
  "you'll": "you you will",  "you'll've": "you you will have",  "you're": "you are", "you've": "you have"}

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
b=strip_html(train["review"][9])
print(b)

In [ ]:
def remove_between_square_brackets(text):
    return re.sub("^a-zA-Z", " ", text)
b=remove_between_square_brackets(train["review"][9])
print(b)
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [ ]:
def expandContractions(words, c_re=c_re):
    def replace(match):
       return cList[match.group(0)]
    return c_re.sub(replace,words)

In [ ]:
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = expandContractions(text)
    text = nltk.word_tokenize(text)
    return text
b=denoise_text(train["review"][9])
print(b)

In [ ]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [ ]:
def to_lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [ ]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [ ]:
def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

In [ ]:
def remove_stopwords(words):
    
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words
def lemmatize_verbs(words):
  
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v'or'n')
        lemmas.append(lemma)
    return lemmas

In [ ]:
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [ ]:
def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

In [ ]:
def final_preprocessing(words):
    words = denoise_text(words)
    words=normalize(words)
    words=lemmatize(words)
    a=[]
    for word in words:
        if ((len(word))>2):
            a.append(word)
    return(" ".join(a))

In [ ]:
print(train["review"][9])
b=final_preprocessing(train["review"][9])
print(b)
print(train.shape)
print ("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []

In [ ]:
for i in range(0,3033) :    
    clean_review = final_preprocessing( train["review"][i] )
    clean_train_reviews.append(clean_review)

In [52]:
                   
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,\
                             lowercase=True, \
                             max_features = 10000) 

In [53]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()
print (train_data_features.shape)

(3033, 10000)


In [54]:
vocab = vectorizer.get_feature_names()
#print (vocab)
#Sum up the counts of each vocabulary word
dist = num.sum(train_data_features, axis=0)
##vocabulary word and the number of times it 
#for tag, count in zip(vocab, dist):
#    print(count, tag)
#
test = pd.read_csv("kdnuggetstest.tsv",na_filter=False,skip_blank_lines=True,header=0, delimiter="\t", \
quoting=3) 
print (test.shape)
clean_test_reviews = [] 

num_reviews=len(test["review"])
print ("CLEANING AND PARSING THE TEST SET MOVIE REVIEWS...\n")
for i in range(0,606):
        clean_review =final_preprocessing( test["review"][i] )
        clean_test_reviews.append(clean_review)
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
#training_model=int(input("PRESS 1 FOR NAIVE BAYES 2 FOR RANDOM FOREST...\n"))

(606, 2)
CLEANING AND PARSING THE TEST SET MOVIE REVIEWS...

PRESS 1 FOR NAIVE BAYES 2 FOR RANDOM FOREST...
1


In [55]:
#Naive Bayes 
bayes = naive_bayes.MultinomialNB()
print ("Training Naive Bayes")

bayess = bayes.fit(train_data_features, train["sentiment"])
result = bayess.predict(test_data_features)

bayes = naive_bayes.MultinomialNB()
rating = bayes.fit(train_data_features,train["id"])

result1 = rating.predict(test_data_features)
result1 = result1.tolist()
c = []
for a in result1:
    b=(a.split("_")[1])
    m = re.findall(r'\d+',b)
    c.append(m[0])
result1n = num.array(c)    

output = pd.DataFrame( data={"id":test["id"],"rating":result1n,"sentiment":result} )
output.to_csv( "finalnaivebayes.csv", index=False, quoting=3,escapechar='\\') 
print("WROTE RESULTS finalnaive_bayes.CSV")
model = pd.read_csv("finalnaivebayes.csv")
count=0
for i in range(0,606):
    if (((int(model["id"][i].split("_")[1]) <= 5) and (model["sentiment"][i] == 0)) or
          ((model["sentiment"][i] == 1) and (int(model["id"][i].split("_")[1])) > 5)):
        count=count+1
a=(count/606)*100
print("accuracy of naive_bayes=")
print(a)
   

Training Naive Bayes
WROTE RESULTS finalnaive_bayes.CSV
accuracy of naive_bayes=
83.33333333333334


In [56]:
#Random Forest
forest = RandomForestClassifier(n_estimators = 100,n_jobs=-1,max_features=100)
forests = forest.fit( train_data_features,train["sentiment"])
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
print("predicting test labels")
result = forests.predict(test_data_features)

forest = RandomForestClassifier(n_estimators = 100,n_jobs=-1,max_features=100)
rating = forest.fit(train_data_features,train["id"])
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result1 = rating.predict(test_data_features)
result1 = result1.tolist()
c = []
for a in result1:
    b=(a.split("_")[1])
    m = re.findall(r'\d+',b)
    c.append(m[0])
result1n = num.array(c)    

output = pd.DataFrame( data={"id":test["id"],"rating":result1n,"sentiment":result} )
output.to_csv( "finalrandom_forest.csv", index=False, quoting=3,escapechar='\\')
print("WROTE RESULTS finalrandom_forest.CSV")
model = pd.read_csv("finalrandom_forest.csv")
count=0
for i in range(0,606):
    if (((int(model["id"][i].split("_")[1]) <= 5) and (model["sentiment"][i] == 0)) or
          ((model["sentiment"][i] == 1) and (int(model["id"][i].split("_")[1])) > 5)):
        count=count+1
a=(count/606)*100
print("accuracy of random forest=")
print(a)

print("%s seconds" %(time.time() - start_time))

predicting test labels
WROTE RESULTS finalrandom_forest.CSV
accuracy of random forest=
81.68316831683168
1088.7025737762451 seconds
